## Install

In [1]:
# !pip install duckdb

In [2]:
# !pip install jupysql pandas matplotlib duckdb-engine

In [13]:
import duckdb
import pandas as pd



In [14]:
test_df = pd.DataFrame({'a' : [1, 2, 3]})


In [15]:
print(duckdb.query("SELECT sum(a) FROM test_df").to_df())

   sum(a)
0     6.0


### Import a CSV

In [17]:
power_plant_raw_df = pd.read_csv("power-plant-generation-history.csv")

In [18]:

query1 = """
SELECT
    *
FROM power_plant_raw_df
limit 5
"""

In [19]:
print(duckdb.sql(query1).df())

      MWh        Date   Plant
0  564337  2019-01-02  Boston
1  507405  2019-01-03  Boston
2  528523  2019-01-04  Boston
3  469538  2019-01-05  Boston
4  474163  2019-01-06  Boston


## Let's try some analytical functions

### The simplest window function is the row_number

In [21]:

query2 = """
SELECT
    "Plant",
    "Date",
    "MWh",
    row_number() OVER (PARTITION BY "Plant" ORDER  BY "Date") AS "Row"
FROM power_plant_raw_df
ORDER BY 1, 2;
"""
print(duckdb.sql(query2).df())

        Plant        Date     MWh  Row
0      Boston  2019-01-02  564337    1
1      Boston  2019-01-03  507405    2
2      Boston  2019-01-04  528523    3
3      Boston  2019-01-05  469538    4
4      Boston  2019-01-06  474163    5
5      Boston  2019-01-07  507213    6
6      Boston  2019-01-08  613040    7
7      Boston  2019-01-09  582588    8
8      Boston  2019-01-10  499506    9
9      Boston  2019-01-11  482014   10
10     Boston  2019-01-12  486134   11
11     Boston  2019-01-13  531518   12
12  Worcester  2019-01-02  118860    1
13  Worcester  2019-01-03  101977    2
14  Worcester  2019-01-04  106054    3
15  Worcester  2019-01-05   92182    4
16  Worcester  2019-01-06   94492    5
17  Worcester  2019-01-07   99932    6
18  Worcester  2019-01-08  118854    7
19  Worcester  2019-01-09  113506    8
20  Worcester  2019-01-10   96644    9
21  Worcester  2019-01-11   93806   10
22  Worcester  2019-01-12   98963   11
23  Worcester  2019-01-13  107170   12


### query to find the highest MWh reading for each plant:

In [22]:
query2 = """
WITH numbered_readings AS (
    SELECT 
        "Plant", 
        "Date", 
        "MWh",
        ROW_NUMBER() OVER (PARTITION BY "Plant" ORDER BY "MWh" DESC) AS "MWhRank"
    FROM 
        power_plant_raw_df
)
SELECT "Plant", "Date", "MWh"
FROM numbered_readings
WHERE "MWhRank" = 1
ORDER BY "Plant";
"""
print(duckdb.sql(query2).df())

       Plant        Date     MWh
0     Boston  2019-01-08  613040
1  Worcester  2019-01-02  118860


In [23]:
#Running Total of MWh
query1 = """
SELECT 
    "Plant", 
    "Date", 
    "MWh",
    SUM("MWh") OVER (PARTITION BY "Plant" ORDER BY "Date") AS "Running_Total_MWh"
FROM 
    power_plant_raw_df
ORDER BY 
    "Plant", 
    "Date";
"""
print(duckdb.sql(query1).df())

        Plant        Date     MWh  Running_Total_MWh
0      Boston  2019-01-02  564337           564337.0
1      Boston  2019-01-03  507405          1071742.0
2      Boston  2019-01-04  528523          1600265.0
3      Boston  2019-01-05  469538          2069803.0
4      Boston  2019-01-06  474163          2543966.0
5      Boston  2019-01-07  507213          3051179.0
6      Boston  2019-01-08  613040          3664219.0
7      Boston  2019-01-09  582588          4246807.0
8      Boston  2019-01-10  499506          4746313.0
9      Boston  2019-01-11  482014          5228327.0
10     Boston  2019-01-12  486134          5714461.0
11     Boston  2019-01-13  531518          6245979.0
12  Worcester  2019-01-02  118860           118860.0
13  Worcester  2019-01-03  101977           220837.0
14  Worcester  2019-01-04  106054           326891.0
15  Worcester  2019-01-05   92182           419073.0
16  Worcester  2019-01-06   94492           513565.0
17  Worcester  2019-01-07   99932           61

In [24]:
#Daily MWh Compared to Plant's Average
query2 = """
SELECT 
    "Plant", 
    "Date", 
    "MWh",
    AVG("MWh") OVER (PARTITION BY "Plant") AS "Avg_Plant_MWh",
    "MWh" - AVG("MWh") OVER (PARTITION BY "Plant") AS "MWh_vs_Avg"
FROM 
    power_plant_raw_df
ORDER BY 
    "Plant", 
    "Date";
"""
print(duckdb.sql(query2).df())

        Plant        Date     MWh  Avg_Plant_MWh    MWh_vs_Avg
0      Boston  2019-01-02  564337  520498.250000  43838.750000
1      Boston  2019-01-03  507405  520498.250000 -13093.250000
2      Boston  2019-01-04  528523  520498.250000   8024.750000
3      Boston  2019-01-05  469538  520498.250000 -50960.250000
4      Boston  2019-01-06  474163  520498.250000 -46335.250000
5      Boston  2019-01-07  507213  520498.250000 -13285.250000
6      Boston  2019-01-08  613040  520498.250000  92541.750000
7      Boston  2019-01-09  582588  520498.250000  62089.750000
8      Boston  2019-01-10  499506  520498.250000 -20992.250000
9      Boston  2019-01-11  482014  520498.250000 -38484.250000
10     Boston  2019-01-12  486134  520498.250000 -34364.250000
11     Boston  2019-01-13  531518  520498.250000  11019.750000
12  Worcester  2019-01-02  118860  103536.666667  15323.333333
13  Worcester  2019-01-03  101977  103536.666667  -1559.666667
14  Worcester  2019-01-04  106054  103536.666667   2517